This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [202]:
# First, import the relevant modules
import requests


In [203]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key='+ API_KEY)


In [204]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

print(r.text)


{"dataset_data":{"limit":1,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2000-06-07","end_date":"2019-01-18","frequency":"daily","data":[["2019-01-18",null,75.8,74.0,75.15,null,null,null,null,null,null]],"collapse":null,"order":null}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [207]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

fetch = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+ API_KEY)

In [208]:
#2. Convert the returned JSON object into a Python dictionary.

fetch_dict = dict(fetch.json()['dataset_data'])


In [209]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.

# Get only the data part and use this in everywhere
all_data = fetch_dict['data']

high = 0    #initialize the high variable to 0
for row in all_data:
    if (isinstance(row[1],float) and row[1]>high):
        high = row[1]

print('This is the highest opening price: ' + str(high))

low = all_data[0][1]    #initialize the low variable to the value of the first row
for row in all_data:
    if (isinstance(row[1],float) and row[1]<low):
        low = row[1]
print('This is the lowest opening price: ' + str(low))

#print(fetch_dict['column_names'])

This is the highest opening price: 53.11
This is the lowest opening price: 34.0


In [210]:
# 4. What was the largest change in any one day (based on High and Low price)?

change=0

for row in all_data:
    if (isinstance(row[2],float) and isinstance(row[3], float)):    # consider only float values
        if abs(row[2]-row[3]) > change:
            change=abs(row[2]-row[3])
            

print('The largest change in one day is: ' + str(round(change,2)))
            


The largest change in one day is: 2.81


In [227]:
# 5 What was the largest change between any two days (based on Closing Price)?
# Assumption: 'any two days' in question are any 2 consecutive trading days.

change=0    #initialize the largest change variable
pre_close=all_data[0][4]    #set the previous day's close value to the first row in the all_data set.

for row in all_data:
    if isinstance(row[4],float):
        if abs(row[4]-pre_close) > change:
            change=abs(row[4]-pre_close)
        pre_close=row[4]

print('The largest change between 2 days (based on Closing Price) is: ' + str(round(change,2)))


The largest change between 2 days (based on Closing Price) is: 2.56


In [229]:
# 6. What was the average daily trading volume during this year?

vol_lst = []    # initialize the list of daily volumes

for row in all_data:
    if isinstance(row[6], float):    #Get the volume,6th index
        vol_lst.append(row[6])
    else:
        vol_lst.append(0.00) # cater for null or none values. If missing we will assume 0. This may skew the result

# Calculate Average volume
avg_vol = sum(vol_lst)/len(vol_lst)

print('Average daily volume during 2017 is: ' + str(round(avg_vol,2)))





Average daily volume during 2017 is: 89124.34


In [226]:
# 7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# Get the middle element from list of yearly volumes to the nearest decimal and subtract 1 to get the index of this value
x = round(len(vol_lst)/2) - 1

# Median value is the middle value (x) in the sorted list of daily volumes
med = sorted(vol_lst)[x]


print('Median trading volume during the year of 2017 is: ' + str(med))


Median trading volume during the year of 2017 is: 76286.0
